In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
num_epochs = 10
batch_size = 32
learning_rate = 0.001

# dataset has PILImage images of range [0,1]
# transform them to Tensors of normalized range of [-1,1]
transform = transforms.Compose(
[transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

# CIFAR10: 60000 32X32 colored images in 10 classes, with 6000 images per class
train_dataset = torchvision.datasets.CIFAR10(root="./data",
                                            train=True,
                                            download=True,
                                            transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root="./data",
                                            train=False,
                                            download=True,
                                            transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                                batch_size=batch_size,
                                                shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                                batch_size=batch_size,
                                                shuffle=False)

classes = {"plane","car","bird","cat","deer","dog","frog","horse","ship","truck"}

In [6]:
# monitor dataset size
print(f"Size of training set: {len(train_dataset)}")
print(f"Number of training batches {len(train_loader)}")
print(f"Size of test set: {len(test_dataset)}")
print(f"Number of test batches {len(test_loader)}")

Size of training set: 50000
Number of training batches 1563
Size of test set: 10000
Number of test batches 313


In [32]:
# Define neural network
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=32,kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2,stride=2)
        self.conv2 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3)
        self.fc1 = nn.Linear(64*4*4,64)
        self.fc2 = nn.Linear(64,10)
        
    def forward(self,x):
        # image shape at the start (3,32,32)
        out = F.relu(self.conv1(x))           # -> (32,30,30)
        out = self.pool(out)                  # -> (32,15,15)
        out = F.relu(self.conv2(out))         # -> (64,13,13)
        out = self.pool(out)                  # -> (64,6,6)
        out = F.relu(self.conv3(out))         # -> (64,4,4)
        out = torch.flatten(out,start_dim=1)  # -> (64x4x4=1024)
        out = F.relu(self.fc1(out))           # -> 64
        out = self.fc2(out)                   # -> 10
        return out

model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate)

n_total_steps = len(train_loader)
num_batches = len(train_loader)
monitor_step_size = 500

for epoch in range(num_epochs):
    for step, (images,labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs,labels)
        
        # Backpropagation and Optimize
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        # Monitor training
        if (step+1) % monitor_step_size == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{step+1}/{n_total_steps}], Loss: {loss.item():.4f}")

Epoch [1/10], Step [500/1563], Loss: 1.7146
Epoch [1/10], Step [1000/1563], Loss: 1.6677
Epoch [1/10], Step [1500/1563], Loss: 1.3876
Epoch [2/10], Step [500/1563], Loss: 0.8687
Epoch [2/10], Step [1000/1563], Loss: 0.8832
Epoch [2/10], Step [1500/1563], Loss: 0.9158
Epoch [3/10], Step [500/1563], Loss: 0.9300
Epoch [3/10], Step [1000/1563], Loss: 1.1777
Epoch [3/10], Step [1500/1563], Loss: 0.8499
Epoch [4/10], Step [500/1563], Loss: 0.8369
Epoch [4/10], Step [1000/1563], Loss: 0.6815
Epoch [4/10], Step [1500/1563], Loss: 0.9567
Epoch [5/10], Step [500/1563], Loss: 0.6486
Epoch [5/10], Step [1000/1563], Loss: 1.1321
Epoch [5/10], Step [1500/1563], Loss: 0.7898
Epoch [6/10], Step [500/1563], Loss: 0.6278
Epoch [6/10], Step [1000/1563], Loss: 0.8523
Epoch [6/10], Step [1500/1563], Loss: 0.7007
Epoch [7/10], Step [500/1563], Loss: 0.6486
Epoch [7/10], Step [1000/1563], Loss: 1.0891
Epoch [7/10], Step [1500/1563], Loss: 0.8607
Epoch [8/10], Step [500/1563], Loss: 0.7549
Epoch [8/10], Step

In [33]:
 # Save the model
path = "./cnn.pth"
torch.save(model.state_dict(),path)

In [38]:
# Evaluate the trained model on the test set
model.eval()
with torch.no_grad():
    num_correct = 0
    num_samples = len(test_dataset)

    for step, (images,labels) in enumerate(test_loader):
        images = images.to(device)
        labels = labels.to(device)
            
        outputs = model(images)
        preds = torch.argmax(outputs,1)
        num_correct += (preds == labels).sum().item()

    # Print results
    acc = num_correct / num_samples
    print(f"Accuracy of the trained model on the test set: {(acc * 100):.2f} %")

Accuracy of the trained model on the test set: 70.78 %


In [ ]:
# load the saved model
loaded_model = ConvNet()
state_dict = torch.load(path)
loaded_model.load_state_dict(state_dict)